In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import io
import pandas as pd
from collections import Counter
import requests
from bs4 import BeautifulSoup as bs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import tika
import pickle
from tika import parser
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 2000000
import re
import pickle

In [ ]:
# run Part 1 only if want to update data; data last updated on 21 March 2021

## Part 1: Download files for 2019-12-19 - Application for Canadian Mainline Contracting (RH-001-2020)

In [ ]:
#get download links for all pdfs in RH-002-2020 proceeding

url_1 = 'https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadResult/3890507' 
cookies = {'RDI-NumberOfRecords' : '200'} 

r = requests.get(url_1, cookies = cookies)
soup = bs(r.text, 'html.parser')
a_tags = [x['href'] for x in soup.find_all('a', {'class' : 'link-wait no-expand'})]
a_tags_full = ['https://apps.cer-rec.gc.ca' + x for x in a_tags]

text_tags = [x.text for x in soup.find_all('a', {'class' : 'link-wait no-expand'})]

df_level1 = pd.DataFrame({'submitter_category' : text_tags, 'folder_link' : a_tags_full})
df_level1 ['folder_link'] = df_level1 ['folder_link'].str.replace('View', 'LoadResult')
df_level1.head()
df_level1.shape

In [ ]:
level2_hrefs = []
for x in df_level1 ['folder_link'][0:2]: #Commission and applicant separate from interveners
    r = requests.get(x, cookies = cookies)
    soup = bs(r.text, 'html.parser')
    a_tags = [x['href'] for x in soup.find_all('a', {'class' : 'link-wait no-expand'})]
    a_tags_full = ['https://apps.cer-rec.gc.ca' + x for x in a_tags]
    a_tags_full = [x.replace('/View', '/LoadResult') for x in a_tags_full]
    text_tags = [x.text for x in soup.find_all('a', {'class' : 'link-wait no-expand'})]
    submitter = [x.text for x in soup.find_all('td', {'class' : 'col-md-2'})]
    date = [x.text for x in soup.find_all('td', {'class' : 'nowrap'})]
    df_hrefs = pd.DataFrame({'folder_name' : text_tags, 'folder_link' : a_tags_full, 'date' : date, 'submitter' : submitter})
    level2_hrefs.append(df_hrefs)
    
df_level2 = pd.concat(level2_hrefs)
df_level2.shape
df_level2.head()
df_level2.tail()

In [ ]:
%%time
# CPU times: user 51 s, sys: 12.9 s, total: 1min 3s
# Wall time: 1h 13min 15s
      
# Go to each download url and convert pdf text to raw string.
# data in list (pdf_strings) saved as pickle file
# raw string by tika for each pdf; 
# next steps - integrate with pdf attributes from PyPDF 

# parallel/threading next step

pdf_strings = []

#go to each url and get the download link
for x, a, b, c in zip(df_level2['folder_link'], df_level2['date'], df_level2['submitter'], df_level2['folder_name']): 
# for x, a, b, c in zip(df_level2['folder_link'][0:2], df_level2['date'][0:2], df_level2['submitter'][0:2], df_level2['folder_name'][0:2]): 
    r = requests.get(x, cookies = cookies)
    soup = bs(r.text, 'html.parser')
    a_tags = [x['href'] for x in soup.find_all('a', {'target' : '_blank'})]
    a_tags_full = ['https://apps.cer-rec.gc.ca' + x for x in a_tags] #download pdf link for tika 
    text_string = [x.text for x in soup.find_all('a', {'target' : '_blank'})]

    for y, z in zip(a_tags_full, text_string): 
        if 'Receipt' not in z: #skip Receipts (each filing grouping has a Receipt)
            try:
                print(y)
                pdf_dict = parser.from_file(y) #start of Tika; type string
                raw_string = pdf_dict['content']
                doc_name = str(z)
                folder_link = str(x)
                date = str(a)
                submitter = str(b)
                folder_name = str(c)
                download_link = str(y)
                dict_list = {'raw string' : raw_string, 'doc_name' : doc_name, 'folder_link' : folder_link, \
                             'date': date, 'submitter' : submitter, 'folder_name' : folder_name, 'download_link': download_link}
                pdf_strings.append(dict_list)
            except:
                print(z)
                pass
        
with open('/Users/margaretskwara/Enbridge Mainline Contracting/Enbridge Mainline Contracting2.txt', 'wb') as fp:
    pickle.dump(pdf_strings, fp)

## Part 2: Open pickle file for data; downloaded 27 April 2021

In [2]:
#load data; remove raw string

with open ('/Users/margaretskwara/Enbridge Mainline Contracting/Enbridge Mainline Contracting2.txt', 'rb') as file:
    pdf_strings = pickle.load(file) # this is a list
len(pdf_strings)

df_sents = pd.DataFrame(pdf_strings)
df_sents['clean string'] = df_sents['raw string'].str.replace('\n', '').str.replace('\t', '').str.replace('\xa0', '')
df_sents = df_sents.drop(['raw string'], axis = 1)
df_sents.to_csv('Enbridge_Mainline_All.csv', index = False)
df_sents.head()

512

,doc_name,folder_link,date,submitter,folder_name,download_link,clean string
0,C12223-1 Commission – PU No. 2 - Enbridge – Ca...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,2021-03-31,Commission | Commission,C12223 Commission | Commission - Procedural Up...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,File OF-Tolls-Group1-E101-2019-02 02 31 M...
1,C12103-1 Commission – Letter – Canadian Shippe...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,2021-03-23,Commission | Commission,C12103 Commission | Commission - Information R...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,File OF-Tolls-Group1-E101-2019-02 02 23 Marc...
2,C12103-2 Commission – IR No. 2 – Canadian Ship...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,2021-03-23,Commission | Commission,C12103 Commission | Commission - Information R...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Attachment to Commission Letter Dated 23 March...
3,C12102-1 Commission – Letter – Suncor Energy -...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,2021-03-23,Commission | Commission,C12102 Commission | Commission - Information R...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,File OF-Tolls-Group1-E101-2019-02 02 23 Marc...
4,C12102-2 Commission – IR No. 2 – Suncor Energy...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,2021-03-23,Commission | Commission,C12102 Commission | Commission - Information R...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Attachment to Commission Letter 23 March 2021 ...


## Part 3: Create sentence tokens

In [3]:
# %%time 
# CPU times: user 13min 30s, sys: 40.9 s, total: 14min 11s
# Wall time: 14min 12s
# https://prrao87.github.io/blog/spacy/nlp/performance/2020/05/02/spacy-multiprocess.html#Option-1:-Sequentially-process-DataFrame-column
# next steps: make data pipeline as per above link    
    
nlp.max_length = 2000000
sents_list = []
# for x, y in zip(df_sents['clean string'][0:4], df_sents['folder_link'][0:4]):
for x, y in zip(df_sents['clean string'], df_sents['download_link']):
    doc = nlp(x)
    sentences = [x.text for x in doc.sents]
    df = pd.DataFrame(sentences, columns = ['sentence'])
    df['download_link'] = str(y)
    sents_list.append(df)

df_sents_new = pd.concat(sents_list)
df_sents_new.head()
df_sents_new.shape

,sentence,download_link
0,File OF-Tolls-Group1-E101-2019-02 02 31 M...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
1,To: All Participants in RH-001-2020 Enbridge...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
2,,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
3,Canadian Mainline Contracting Application Hear...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
4,"On 23 February 2021, the Commission of the Can...",https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...


(164496, 2)

## Part 4: NLP basics on entire record

In [ ]:
%%time 
#Wall time: 18min 51s
tokens_list = []
ents_list = []
nouns_list = []
nlp.max_length = 2000000

for x , y in zip(df_sents_new['sentence'].to_list(), df_sents_new['link'].to_list()): #testing all
# for x , y in zip(df['text_string'].to_list()[0:2], df['link'].to_list()[0:2]): #testing small
    doc = nlp(x)
    
    tokens = [(x.text, x.lemma_, x.pos_, x.tag_, x.dep_, x.shape_, x.is_alpha, x.is_stop) for x in doc]
    df_tokens = pd.DataFrame(tokens, columns = ['token', 'lemma', 'pos', 'tag', 'dep', 'shape_', 'is_alpha', 'is_stop'])
    df_tokens['link'] = y
    tokens_list.append(df_tokens)
    
    ents = [(x.text, x.start_char, x.end_char, x.label_) for x in doc.ents]
    df_ents = pd.DataFrame(ents, columns = ['token', 'start_char', 'end_char', 'ent_label'])
    df_ents['link'] = y
    df_ents['sentence'] = x
    ents_list.append(df_ents)
    
    nouns = [(x.text, x.root.text, x.root.dep_, x.root.head.text) for x in doc.noun_chunks]
    df_nouns = pd.DataFrame(nouns, columns = ['noun_chunks', 'root_text', 'root_dep', 'root_head'])
    df_nouns['link'] = y 
    nouns_list.append(df_nouns)

In [ ]:
df_tokens_all = pd.concat(tokens_list)
df_tokens_all.head()
df_tokens_all = df_tokens_all[df_tokens_all['is_stop'] == False] #remove stop words
df_tokens_all = df_tokens_all[df_tokens_all['is_alpha'] == True] #remove 
df_tokens_all.to_csv('Enbridge_Mainline_tokens.csv', index = False)

In [ ]:
df_ents_all = pd.concat(ents_list)
df_ents_all['explained'] = df_ents_all['ent_label'].apply(lambda x: spacy.explain(x))
df_ents_all.head()
df_ents_all.shape
df_ents_all.to_csv('Enbridge_Mainline_ents.csv', index = False)

In [ ]:
df_nouns_all = pd.concat(nouns_list)
df_nouns_all.head()
df_nouns_all.to_csv('Enbridge_Mainline_nouns.csv', index = False)
df_nouns_all.shape

## Part 5: NLP basics on sentence-specific topics

In [33]:
df_sents_new.head()
df_sents_new.shape

,sentence,download_link
0,File OF-Tolls-Group1-E101-2019-02 02 31 M...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
1,To: All Participants in RH-001-2020 Enbridge...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
2,,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
3,Canadian Mainline Contracting Application Hear...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...
4,"On 23 February 2021, the Commission of the Can...",https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...


(164496, 2)

In [34]:
pd.options.mode.chained_assignment = None  # default='warn'
# create dfs based on topic
# next steps: explore regex, + or - ten sentences in index to expand scope
# remove not sentences ex. classify is sentence is not sentence 

df_public_interest = df_sents_new[df_sents_new['sentence'].str.contains('public interest')]
df_public_interest['type'] = 'public interest'

df_competition = df_sents_new[df_sents_new['sentence'].str.contains('competitiveness|competitive|competition')]#.reset_index(drop = True)
df_competition['type'] = 'competition'  # df.loc[:, 'newCol'] = df['Month']*2 still error hence chained_assignment = None

df_reconciliation = df_sents_new[df_sents_new['sentence'].str.contains('Indigenous|First Nation|econciliation')]#.reset_index(drop = True)
df_reconciliation['type'] = 'Reconciliation'

df_market_power = df_sents_new[df_sents_new['sentence'].str.contains('market power')]#.reset_index(drop = True)
df_market_power['type'] = 'market power'

df_tolling = df_sents_new[df_sents_new['sentence'].str.contains('tolling principles|just and reasonable|economic efficiency|market signal|discriminatory|cross-subsidization')]#.reset_index(drop = True)
df_tolling['type'] = 'tolling'

df_risk = df_sents_new[df_sents_new['sentence'].str.contains('fundamental risk')]#.reset_index(drop = True)
df_risk['type'] = 'risk'

df_keystone = df_sents_new[df_sents_new['sentence'].str.contains('eystone|KXL')]#.reset_index(drop = True)
df_keystone['type'] = 'Keystone'

df_liquefied = df_sents_new[df_sents_new['sentence'].str.contains('liquefied|LNG')]#.reset_index(drop = True)
df_liquefied['type'] = 'LNG'


df_trans_mountain = df_sents_new[df_sents_new['sentence'].str.contains('Trans Mountain|TMX')]#.reset_index(drop = True)
df_trans_mountain['type'] = 'Trans Mountain'

df_sentences_type = pd.concat([df_public_interest, df_reconciliation, df_competition, df_liquefied, df_market_power, df_tolling, df_risk, df_keystone, df_trans_mountain])
df_sentences_type.shape
df_sentences_type.to_csv('Enbridge_Mainline_All_sentspecial.csv', index = False)
df_sentences_type.tail()

(4178, 3)

,sentence,download_link,type
453,"Barrels of CrudePetroleum per day, where the e...",https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Trans Mountain
439,Application for Canadian Mainline Contracting ...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Trans Mountain
496,Feedstock Impairment14.4(a) Subject to the pro...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Trans Mountain
518,-and has the effect of reducing the volumes of...,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Trans Mountain
519,"Barrels of CrudePetroleum per day, where the e...",https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Trans Mountain


In [35]:
%%time 
# CPU times: user 37.8 s, sys: 197 ms, total: 38 s
# Wall time: 38.1 s

tokens_list = []
ents_list = []
nouns_list = []
for x , y, z in zip(df_sentences_type['sentence'].to_list(), df_sentences_type['download_link'].to_list(), df_sentences_type['type'].to_list()):
# for x , y in zip(df['text_string'].to_list()[0:2], df['link'].to_list()[0:2]): #testing small
    doc = nlp(x)
    
    tokens = [(x.text, x.lemma_, x.pos_, x.tag_, x.dep_, x.shape_, x.is_alpha, x.is_stop) for x in doc]
    df_tokens = pd.DataFrame(tokens, columns = ['token', 'lemma', 'pos', 'tag', 'dep', 'shape_', 'is_alpha', 'is_stop'])
    df_tokens['download_link'] = y
    df_tokens['type'] = z
    df_tokens['sentence'] = x
    tokens_list.append(df_tokens)
    
    ents = [(x.text, x.start_char, x.end_char, x.label_) for x in doc.ents]
    df_ents = pd.DataFrame(ents, columns = ['token', 'start_char', 'end_char', 'ent_label'])
    df_ents['download_link'] = y
    df_ents['sentence'] = x
    df_ents['type'] = z
    ents_list.append(df_ents)
    
    nouns = [(x.text, x.root.text, x.root.dep_, x.root.head.text) for x in doc.noun_chunks]
    df_nouns = pd.DataFrame(nouns, columns = ['noun_chunks', 'root_text', 'root_dep', 'root_head'])
    df_nouns['download_link'] = y 
    df_nouns['type'] = z
    df_nouns['sentence'] = x
    nouns_list.append(df_nouns)

CPU times: user 38.9 s, sys: 303 ms, total: 39.2 s
Wall time: 39.3 s


In [36]:
df_tokens_all = pd.concat(tokens_list)
df_tokens_all.head()
df_tokens_all = df_tokens_all[df_tokens_all['is_stop'] == False] #remove stop words
df_tokens_all = df_tokens_all[df_tokens_all['is_alpha'] == True] #remove 
df_tokens_all.to_csv('Enbridge_Mainline_tokens_sentspecial.csv', index = False)

,token,lemma,pos,tag,dep,shape_,is_alpha,is_stop,download_link,type,sentence
0,d,d,X,LS,meta,x,True,False,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,public interest,"d) Explain which crude oil price impacts, if a..."
1,),),PUNCT,-RRB-,punct,),False,False,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,public interest,"d) Explain which crude oil price impacts, if a..."
2,Explain,explain,VERB,VB,ROOT,Xxxxx,True,False,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,public interest,"d) Explain which crude oil price impacts, if a..."
3,which,which,DET,WDT,det,xxxx,True,True,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,public interest,"d) Explain which crude oil price impacts, if a..."
4,crude,crude,ADJ,JJ,amod,xxxx,True,False,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,public interest,"d) Explain which crude oil price impacts, if a..."


In [37]:
df_ents_all = pd.concat(ents_list)
df_ents_all['explained'] = df_ents_all['ent_label'].apply(lambda x: spacy.explain(x))
df_ents_all.head()
df_ents_all.shape
df_ents_all.to_csv('Enbridge_Mainline_ents_sentspecial.csv', index = False)

,token,start_char,end_char,ent_label,download_link,sentence,type,explained
0,Canadian,130,138,NORP,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,"d) Explain which crude oil price impacts, if a...",public interest,Nationalities or religious or political groups
0,1 February 2021,38,53,DATE,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Attachment to Commission Letter dated 1 Februa...,public interest,Absolute or relative dates or periods
1,5,59,60,CARDINAL,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Attachment to Commission Letter dated 1 Februa...,public interest,Numerals that do not fall under another type
2,Canadian,199,207,NORP,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Attachment to Commission Letter dated 1 Februa...,public interest,Nationalities or religious or political groups
0,Canadian,130,138,NORP,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,"d) Explain which crude oil price impacts, if a...",public interest,Nationalities or religious or political groups


(16799, 8)

In [38]:
df_nouns_all = pd.concat(nouns_list)
df_nouns_all.head()
df_nouns_all.to_csv('Enbridge_Mainline_nouns_sentspecial.csv', index = False)
df_nouns_all.shape

,noun_chunks,root_text,root_dep,root_head,download_link,type,sentence
0,which crude oil price impacts,impacts,dobj,Explain,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,public interest,"d) Explain which crude oil price impacts, if a..."
1,your business,business,dobj,benefit,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,public interest,"d) Explain which crude oil price impacts, if a..."
2,the Canadian public interest,interest,dobj,benefit,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,public interest,"d) Explain which crude oil price impacts, if a..."
0,Attachment,Attachment,nsubj,dated,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,public interest,Attachment to Commission Letter dated 1 Februa...
1,Commission Letter,Letter,pobj,to,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,public interest,Attachment to Commission Letter dated 1 Februa...


(37746, 7)

## Testing

In [43]:
items = ['fundamental risk']

# [df_sents_new['sentence']
tolling_list = [y for y in df_sents_new['sentence'].to_list() if any(x in y for x in items)]
len(tolling_list)
tolling_list[0:20]

52

['In reference iv), Enbridge submits that an analysis on fundamental risk would require a detailed understanding of what alternative form of regulation and what approved level of return would prevail in the absence of the Mainline Contract Offering, and would involve extensive speculation and likely not be reliable.',
 'In reference vi), Enbridge submits that the manifestation of fundamental risk would occur if the market could not sustain cost-based tolls, and is a risk  that the pipeline must bear under the Canada Energy Regulator’s (CER) guidance on cost-based tolls.',
 'Discuss whether the possible 100% uncommitted service structure tolls, under a 2,250 kpbd throughput scenario, as provided in reference v), could, in Enbridge’s view, result in: b.1) uncompetitive toll levels; b.2) a toll spiral; and/or b.3) the materialization of fundamental risk.',
 'e) Discuss whether the range of possible 100% uncommitted service structure tolls provided in response to c) could, in Enbridge’s vi

In [42]:
items = ['uncompetitive']

# [df_sents_new['sentence']
tolling_list = [y for y in df_sents_new['sentence'].to_list() if any(x in y for x in items)]
len(tolling_list)
tolling_list[0:20]

18

['Discuss whether the possible 100% uncommitted service structure tolls, under a 2,250 kpbd throughput scenario, as provided in reference v), could, in Enbridge’s view, result in: b.1) uncompetitive toll levels; b.2) a toll spiral; and/or b.3) the materialization of fundamental risk.',
 'e) Discuss whether the range of possible 100% uncommitted service structure tolls provided in response to c) could, in Enbridge’s view, result in: e.1) uncompetitive toll levels; e.2) a toll spiral; and/or e.3) the materialization of fundamental risk.',
 'No.:  OF-Tolls-Group-E101-2019-02 02Reply Evidence of Concentric Energy Advisors Page 47 of 116 https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4028355 45  market conditions has been that it would be unreasonable to expect any pipeline owner to 1 commit hundreds of millions, or billions, of dollars for the construction of new capacity if 2 the pipeline’s existing capacity is required to operate on an uncompetitive basis in an 3 uncertain environment.